Author: Irsyad Adam

- Neo4j Python Driver Manual: https://neo4j.com/docs/api/python-driver/current/
- API Documentation: https://neo4j.com/docs/api/python-driver/current/api.html



In [60]:
import pandas as pd
import numpy as np

from tqdm import tqdm
from neo4j import GraphDatabase
from urllib import request

In [70]:
members = pd.read_csv('zach_karate_club/members.csv')
interactions = pd.read_csv('zach_karate_club/interactions.csv')

In [71]:
members.head()

,Id,Club,Age
0,0,Mr. Hi,44
1,1,Mr. Hi,37
2,2,Mr. Hi,37
3,3,Mr. Hi,40
4,4,Mr. Hi,30


In [72]:
interactions.head()

,Src,Dst,Weight
0,0,1,0.043591
1,0,2,0.282119
2,0,3,0.370293
3,0,4,0.730570
4,0,5,0.821187


#### Connecting to the DBMS

If there is any authentification error after you run any queries, then your username and password are wrong

In [64]:
# Seen from :server status
uri = "bolt://localhost:7687"

# default user for graph database is neo4j
# auth = ("neo4j", "password")
auth = ("neo4j","quickstart")

driver = GraphDatabase.driver(uri = uri, auth = auth)
driver.verify_connectivity()

C:\Users\irsya\AppData\Local\Temp/ipykernel_16944/1292716654.py:9: ExperimentalWarning: The configuration may change in the future.
  driver.verify_connectivity()


'Neo4j/4.4.4'

#### Creating Constraints

Constraints ensure that there are no duplicate nodes in the graph.

Availiable Constraints:
- **Unique node property constraints**
- Node property existence constraints
- Relationship property existence constraints
- Node key constraints


NOTE: Constraints are also unique, so once you create constraints, recreating that same constraint will result in an error

    CALL db.constraints #see existing constraints

    DROP CONSTRAINT "constraint_name" #drop a constraint

    CALL apoc.schema.assert({}, {}); #delete all constraints


In [65]:
query = "CREATE CONSTRAINT ON (p:Person) ASSERT p.id  IS UNIQUE"       

info = driver.session().run(query)
response = driver.session().run("CALL db.constraints").data()
response

[{'name': 'constraint_bb87716',
  'description': 'CONSTRAINT ON ( person:Person ) ASSERT (person.id) IS UNIQUE',
  'details': "Constraint( id=6, name='constraint_bb87716', type='UNIQUENESS', schema=(:Person {id}), ownedIndex=5 )"}]

#### Importing Data

##### Creating Nodes

In [66]:
def create_nodes(tx, id, club, age) -> None:
    """
    parameters of create_nodes are metadata for nodes
    """
    query = """
            MERGE (p:Person {id: $id, club: $club, age: $age})
            """
    tx.run(query, id = id, club = club, age = age)

In [67]:
for i in tqdm(members.itertuples(), desc = "Deploying Person Nodes"):
    (_, id, club, age) = i
    driver.session().write_transaction(create_nodes, id, club, age)

print("Success", flush=True)

Deploying Person Nodes: 34it [00:00, 104.61it/s]

Success


##### Creating Relationships

In [68]:
def create_relationships(tx, id1, id2, weight) -> None:
    """
    Args:
        id1 is the id of first node
        id2 is id of second 
        NOTE: SRC-->DEST
    """
    query = """
            MATCH (p:Person {id: $id1})
            MATCH (n:Person {id: $id2})
            MERGE (p)-[:WEIGHT {weight: $weight}]->(n)
            """
    tx.run(query, id1 = id1, id2 = id2, weight = weight)

In [69]:
for i in tqdm(interactions.itertuples(), desc = "Deploying Person Nodes"):
    (_, id1, id2, weight) = i
    driver.session().write_transaction(create_relationships, id1, id2, weight)

print("Success", flush=True)

Deploying Person Nodes: 156it [00:01, 118.26it/s]

Success
